In [3]:
from __future__ import print_function
import nltk, re, pprint
from nltk import grammar, parse, load_parser

In [16]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}
sent = "Kim chased Lee"
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}
def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            print(fs)
            return fs
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])

verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']
print(chase)
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print("%-5s => %s" % (k, verb[k]))
print(verb[k]) 

{'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
{'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase', 'AGT': 'sbj', 'PAT': 'obj'}
{'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}
{'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase', 'AGT': 'k', 'PAT': 'l'}
ORTH  => chased
REL   => chase
AGT   => k
PAT   => l
l


In [18]:
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print("%s => %s" % (k, verb[k]))

ORTH => chased
REL => chase
AGT => k
PAT => l


In [17]:
#the % character informs python it will have to substitute something to a token
#the s character informs python the token will be a string
#the 5 (or whatever number you wish) informs python to pad the string with spaces up to 5 characters.

1. What constraints are required to correctly parse word sequences like I am happy and she is happy but not *you is happy or *they am happy? Implement two solutions for the present tense paradigm of the verb be in English, first taking Grammar
(8) as your starting point, and then taking Grammar (20) as the starting point.

In [28]:
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


In [30]:
grammar = load_parser('file:Cp9_1.fcfg')

In [31]:
tokens1 = 'I am happy'.split()

In [32]:
tokens2 = 'she is happy'.split()

In [34]:
for tree in grammar.parse(tokens1):
    print(tree)

(S[]
  (NP_SG_1[] (Pro_SG_1[] I))
  (VP_SG_1[] (Cop_SG_1[] am) (Adj[] happy)))


In [35]:
for tree in grammar.parse(tokens2):
    print(tree)

(S[]
  (NP_SG_3[] (Pro_SG_3[] she))
  (VP_SG_3[] (Cop_SG_3[] is) (Adj[] happy)))


In [42]:
grammar_a = load_parser('file:Cp9_1_a.fcfg')
for tree in grammar_a.parse(tokens1):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=1]] (Pro[AGR=[NUM='sg', PER=1]] I))
  (VP[AGR=[NUM='sg', PER=1]]
    (Cop[AGR=[NUM='sg', PER=1]] am)
    (Adj[] happy)))


In [44]:
tokens3 = 'you are happy'.split()
for tree in grammar_a.parse(tokens3):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=2]] (Pro[AGR=[NUM='sg', PER=2]] you))
  (VP[AGR=[NUM='sg', PER=2]]
    (Cop[AGR=[NUM='sg', PER=2]] are)
    (Adj[] happy)))
(S[]
  (NP[AGR=[NUM='pl', PER=2]] (Pro[AGR=[NUM='pl', PER=2]] you))
  (VP[AGR=[NUM='pl', PER=2]]
    (Cop[AGR=[NUM='pl', PER=2]] are)
    (Adj[] happy)))


2.  Develop a variant of grammar in Example 9-1 that uses a feature COUNT to make the distinctions shown here:
(56) a. The boy sings.
b. *Boy sings.
(57) a. The boys sing.
b. Boys sing.
(58) a. The water is precious.
b. Water is precious.

In [2]:
# COUNT Nouns which can be made plural are +COUNT, e.g. dog. Nouns which are –COUNT include water (in its usual sense), mankind or sincerity.

In [101]:
sent1 = "The boy sings".lower().split()
sent2 = "The boys sing".lower().split()
sent3 = "Boys sing".lower().split()
sent4 = "The water is precious".lower().split()
sent5 = "Water is precious".lower().split()
sent6 = "Boy sings".lower().split()
grammar_a = load_parser('file:Cp9_2a.fcfg')

In [46]:
for tree in grammar_a.parse(sent1):
    print(tree)

(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg', +count] boy))
  (VP[NUM='sg', TENSE='pres'] (IV[NUM='sg', TENSE='pres'] sings)))


In [47]:
for tree in grammar_a.parse(sent2):
    print(tree)

(S[]
  (NP[NUM='pl'] (Det[] the) (N[NUM='pl', +count] boys))
  (VP[NUM='pl', TENSE='pres'] (IV[NUM='pl', TENSE='pres'] sing)))


In [102]:
for tree in grammar_a.parse(sent6):
    print(tree)

3. Write a function subsumes() that holds of two feature structures fs1 and fs2 just in case fs1 subsumes fs2.

In [104]:
# dont understand the task

4.  Modify the grammar illustrated in (30) to incorporate a BAR feature for dealing with phrasal projections.

In [51]:

VP[TENSE=?t, NUM=?n] -> V[SUBCAT=intrans, TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=trans, TENSE=?t, NUM=?n] NP
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=clause, TENSE=?t, NUM=?n] SBar

SBar -> Comp S
Comp -> 'that'

S -> N[BAR=1] V[BAR=2] 
N[BAR=1] -> N[BAR=0] 
V[BAR=2] -> V[BAR=1] N[BAR=1] 
V[BAR=1] -> V[BAR=0]


SyntaxError: invalid syntax (<ipython-input-51-56a03c099ea7>, line 1)

5.  Modify the German grammar in Example 9-4 to incorporate the treatment of subcategorization presented in Section 9.3.

In [109]:
% start S
 # Grammar Productions
 S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
 NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
 NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
 VP[AGR=?a] -> IV[[AGR=?a], SUBCAT=intrans]
 VP[AGR=?a] -> TV[[OBJCASE=?c, AGR=?a], SUBCAT=trans] NP[CASE=?c]
 # Lexical Productions
 # Singular determiners
 # masc
 Det[CASE=nom, AGR=[GND=masc,PER=3,NUM=sg]] -> 'der'
 Det[CASE=dat, AGR=[GND=masc,PER=3,NUM=sg]] -> 'dem'
 Det[CASE=acc, AGR=[GND=masc,PER=3,NUM=sg]] -> 'den'
 # fem
 Det[CASE=nom, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die'
 Det[CASE=dat, AGR=[GND=fem,PER=3,NUM=sg]] -> 'der'
 Det[CASE=acc, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die'
 # Plural determiners
 Det[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'die'
 Det[CASE=dat, AGR=[PER=3,NUM=pl]] -> 'den'
 Det[CASE=acc, AGR=[PER=3,NUM=pl]] -> 'die'
 # Nouns
 N[AGR=[GND=masc,PER=3,NUM=sg]] -> 'Hund'
 N[CASE=nom, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
 N[CASE=dat, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunden'
 N[CASE=acc, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
 N[AGR=[GND=fem,PER=3,NUM=sg]] -> 'Katze'
 N[AGR=[GND=fem,PER=3,NUM=pl]] -> 'Katzen'
 # Pronouns
 PRO[CASE=nom, AGR=[PER=1,NUM=sg]] -> 'ich'
 PRO[CASE=acc, AGR=[PER=1,NUM=sg]] -> 'mich'
 PRO[CASE=dat, AGR=[PER=1,NUM=sg]] -> 'mir'
 PRO[CASE=nom, AGR=[PER=2,NUM=sg]] -> 'du'
 PRO[CASE=nom, AGR=[PER=3,NUM=sg]] -> 'er' | 'sie' | 'es'
 PRO[CASE=nom, AGR=[PER=1,NUM=pl]] -> 'wir'
 PRO[CASE=acc, AGR=[PER=1,NUM=pl]] -> 'uns'
 PRO[CASE=dat, AGR=[PER=1,NUM=pl]] -> 'uns'
 PRO[CASE=nom, AGR=[PER=2,NUM=pl]] -> 'ihr'
 PRO[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'sie'
 # Verbs
 IV[AGR=[NUM=sg,PER=1], SUBCAT=intrans] -> 'komme'
 IV[AGR=[NUM=sg,PER=2], SUBCAT=intrans] -> 'kommst'
 IV[AGR=[NUM=sg,PER=3], SUBCAT=intrans] -> 'kommt'
 IV[AGR=[NUM=pl, PER=1], SUBCAT=intrans] -> 'kommen'
 IV[AGR=[NUM=pl, PER=2], SUBCAT=intrans] -> 'kommt'
 IV[AGR=[NUM=pl, PER=3], SUBCAT=intrans] -> 'kommen'
 TV[OBJCASE=acc, AGR=[NUM=sg,PER=1], SUBCAT=trans] -> 'sehe' | 'mag'
 TV[OBJCASE=acc, AGR=[NUM=sg,PER=2], SUBCAT=trans] -> 'siehst' | 'magst'
 TV[OBJCASE=acc, AGR=[NUM=sg,PER=3], SUBCAT=trans] -> 'sieht' | 'mag'
 TV[OBJCASE=dat, AGR=[NUM=sg,PER=1], SUBCAT=trans] -> 'folge' | 'helfe'
 TV[OBJCASE=dat, AGR=[NUM=sg,PER=2], SUBCAT=trans] -> 'folgst' | 'hilfst'
 TV[OBJCASE=dat, AGR=[NUM=sg,PER=3], SUBCAT=trans] -> 'folgt' | 'hilft'
 TV[OBJCASE=acc, AGR=[NUM=pl,PER=1], SUBCAT=trans] -> 'sehen' | 'moegen'
 TV[OBJCASE=acc, AGR=[NUM=pl,PER=2], SUBCAT=trans] -> 'sieht' | 'moegt'
 TV[OBJCASE=acc, AGR=[NUM=pl,PER=3], SUBCAT=trans] -> 'sehen' | 'moegen'
 TV[OBJCASE=dat, AGR=[NUM=pl,PER=1], SUBCAT=trans] -> 'folgen' | 'helfen'
 TV[OBJCASE=dat, AGR=[NUM=pl,PER=2], SUBCAT=trans] -> 'folgt' | 'helft'
 TV[OBJCASE=dat, AGR=[NUM=pl,PER=3], SUBCAT=trans] -> 'folgen' | 'helfen'

IndentationError: unexpected indent (<ipython-input-109-67d7c1847e5c>, line 3)

6. Develop a feature-based grammar that will correctly describe the following Spanish noun phrases:
(59) un cuadro hermos-o
INDEF.SG.MASC picture beautiful-SG.MASC
‘a beautiful picture’
(60) un-os cuadro-s hermos-os
INDEF-PL.MASC picture-PL beautiful-PL.MASC
‘beautiful pictures’

(61) un-a cortina hermos-a
INDEF-SG.FEM curtain beautiful-SG.FEM
‘a beautiful curtain’
(62) un-as cortina-s hermos-as
INDEF-PL.FEM curtain beautiful-PL.FEM
‘beautiful curtains’

In [38]:
grammar_a = load_parser('file:Cp9_6b.fcfg')
orac1 = 'un cuadro hermoso'.split()
orac2 = 'unos cuadros hermosos'.split()
orac3 = 'una cortina hermosa'.split()
orac4 = 'unas cortinas hermosas'.split()

In [39]:
for tree in grammar_a.parse(orac2):
    print(tree)

(NP[]
  (Det[ARG=[GND='MASC', NUM='pl']] unos)
  (NP[ARG=[GND='MASC', NUM='pl']]
    (N[ARG=[GND='MASC', NUM='pl']] cuadros)
    (ADJ[ARG=[GND='MASC', NUM='pl']] hermosos)))


7. ◑ Develop a wrapper for the earley_parser so that a trace is only printed if the
input sequence fails to parse.

8. ◑ Consider the feature structures shown in Example 9-5.
Example 9-5. Exploring feature structures.
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")
Work out on paper what the result is of the following unifications. (Hint: you might find it useful to draw the graph structures.)
a. fs1 and fs2
b. fs1 and fs3
c. fs4 and fs5
d. fs5 and fs6
e. fs5 and fs7
f. fs8 and fs9
g. fs8 and fs10
Check your answers using NLTK.

In [40]:
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")

In [47]:
print(fs8.unify(fs10))

[         [ B = 'b'  ] ]
[ A = (1) [ D = 'b'  ] ]
[         [ E -> (1) ] ]
[         [ G = 'b'  ] ]
[                      ]
[ C -> (1)             ]


11. ◑ Extend the German grammar in Example 9-4 so that it can handle so-called verbsecond
structures like the following:
(63) Heute sieht der Hund die Katze.

12. ◑ Seemingly synonymous verbs have slightly different syntactic properties (Levin,
1993). Consider the following patterns of grammaticality for the verbs loaded,
filled, and dumped. Can you write grammar productions to handle such data?
(64) a. The farmer loaded the cart with sand
b. The farmer loaded sand into the cart
c. The farmer filled the cart with sand
d. *The farmer filled sand into the cart
e. *The farmer dumped the cart with sand
f. The farmer dumped sand into the cart

In [88]:
sent12a = "The farmer loaded the cart with sand".lower().split()
sent12b = "The farmer loaded sand into the cart".lower().split()
sent12c = "The farmer filled the cart with sand".lower().split()
sent12d = "The farmer filled sand into the cart".lower().split()
sent12e = 'The farmer dumped the cart with sand'.lower().split()
sent12f = 'The farmer dumped sand into the cart'.lower().split()
grammar_12 = load_parser('file:Cp9_12d.fcfg')

In [90]:
for tree in grammar_12.parse(sent12e):
    print(tree)

(S[]
  (NP[NUM='sg'] (Det[NUM='sg'] the) (N[NUM='sg', +count] farmer))
  (VP[AGR=?a]
    (TV[OBJCASE='f2', TENSE='past'] dumped)
    (NP[NUM='sg'] (Det[NUM='sg'] the) (N[NUM='sg', +count] cart))
    (PP[NUM=?n] (P[OBJCASE='f1'] with) (N[NUM='sg', -count] sand))))
